<a href="https://colab.research.google.com/github/OswaldoLopezAlcaraz/TI_1_Practica4/blob/main/TI_1_Practica4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica 4

**Nombre:** Oswaldo López Alcaraz

**e-mail:** oswaldo.lopez5103@alumnos.udg.mx

# MODULES

In [1]:
import panel as pn
import panel.widgets as pnw

pn.extension('plotly')

import pandas as pd
import numpy as np

import plotly.graph_objects as go

import math
from scipy.stats import wrapcauchy
from scipy.stats import levy_stable
from pandas._libs.tslibs import vectorized

# CLASSES

In [2]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y

    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)

    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)

    # rotate vector
    def rotated(self, angle):
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

# FUNCTIONS

In [3]:
###################### Function to create Brownian walks #######################
def bm_2d(n_steps=1000, speed=6, s_x_pos=0, s_y_pos=0):
  """
  Summary:
    This function generates a Brownian Walk.
  Arguments:
    n_steps: number of steeps that comprises the entire walk.
    speed: number of units per steps.
    s_pos: coordinates (x and y ) where the walk starts.
  Returns:
    BM_2d_df: dataframe with the walk in form of coordinates.
  """
  # Init velocity vector
  velocity = Vec2d(speed, 0)

  BM_2d_df = pd.DataFrame(columns = ['x_pos','y_pos'])
  temp_df = pd.DataFrame([{'x_pos': s_x_pos, 'y_pos': s_y_pos}])

  BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

  for i in range(n_steps-1):
    turn_angle = np.random.uniform(low=-np.pi, high=np.pi)
    velocity = velocity.rotated(turn_angle)

    temp_df = pd.DataFrame([{'x_pos':BM_2d_df.x_pos[i]+velocity.x, 'y_pos':BM_2d_df.y_pos[i]+velocity.y}])
    BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

  return BM_2d_df


################## Function to create Correlated Random Walks ##################
def crw_2d(n_steps=1000, speed=6, s_x_pos=0, s_y_pos=0, crw_exp=0.3):
  """
  Summary:
    This function generates a Correlated Random Walk.
  Arguments:
    n_steps: number of steeps that comprises the entire walk
    speed: number of units per steps
    s_pos: coordinates (x and y ) where the walk starts.
    crw_exp:  exponent used in the Cauchy distribution
  Returns:
    CRW_2d_df: dataframe with the walk in form of coordinates.
  """
  # Init of velocity vector
  velocity = Vec2d(speed,0)

  CRW_2d_df = pd.DataFrame( columns= ['x_pos','y_pos'])
  temp_df = pd.DataFrame([{'x_pos':s_x_pos,'y_pos': s_y_pos},])
  CRW_2d_df = pd.concat([CRW_2d_df, temp_df], ignore_index=True)

  # Brownian walker in motion and assignation of directions
  for i in range(1, n_steps):

    # Determination of angle using cauchy distribution
    turn_angle = wrapcauchy.rvs(crw_exp)
    velocity = velocity.rotated(turn_angle)

    # Motion of Brownian walker
    temp_df = pd.DataFrame([{'x_pos':CRW_2d_df.x_pos[i-1]+velocity.x,
                           'y_pos':CRW_2d_df.y_pos[i-1]+velocity.y,}])
    CRW_2d_df = pd.concat([CRW_2d_df,temp_df], ignore_index=True)

  return CRW_2d_df


########## Function to obtain Euclidian distance between two points ############
def my_distance_function(origin, destination):
  """
  Arguments:
    origin: x & y coordinates that represent the origin of a trajectory
    destination: x & y coordinate representing the destination of a trajectory
  Returns:
    Euclidiean distance between two points in the form of int or float.
  """
  return math.sqrt(((destination.x_pos - origin.x_pos)**2) +
                    (( destination.y_pos - origin.y_pos)**2) )

################# Function to get Squared Mean Displacement ####################
def smd(motion_df):
  """
  Arguments:
    motion_df: dataframe with coordinates that represents a certain type of displacement.
  Returns:
    dataframe of length motion.df.shape[0] - 1 containing SMD in for each step
  """
  smd = pd.DataFrame(columns=['MSD'])

  for n in range(1,motion_df.shape[0]):

    displacement_vec = np.array([(my_distance_function(motion_df.iloc[i-n], motion_df.iloc[i]))**2
                              for i in range(n, motion_df.shape[0],1)])
    tmp_df = pd.DataFrame([{'MSD':np.mean(displacement_vec)}])
    smd = pd.concat([smd,tmp_df], ignore_index=True)

  return smd


##################### Function to create Brownian walks ########################
def turning_angle(pos_a, pos_b, pos_c):
    """
    Arguments:
        pos_a: First position coordinates
        pos_b: Second position coordinates
        pos_c: Third position coordinates
    Returns:
        theta: Turning angle
    """
    vec_ab = np.array([pos_b.x_pos - pos_a.x_pos , pos_b.y_pos - pos_a.y_pos ])
    norm_ab = np.linalg.norm(vec_ab)

    vec_bc = np.array([pos_c.x_pos - pos_b.x_pos , pos_c.y_pos - pos_b.y_pos ])
    norm_bc = np.linalg.norm(vec_bc)

    dot_p =np.dot(vec_ab , vec_bc)

    cos_theta = dot_p/(norm_ab*norm_bc+np.finfo(float).eps)

    # Angle orientation
    cross_p = np.cross(vec_ab, vec_bc)
    orient = np.sign(cross_p)
    if orient == 0:
        orient = 1

    theta = np.arccos(np.around(cos_theta,4)) * orient
    return theta


##################### Function to create Brownian walks Vectorization########################
def turning_angle2(pos_a, pos_b, pos_c):
    """
    Arguments:
        pos_a: First position coordinates
        pos_b: Second position coordinates
        pos_c: Third position coordinates
    Returns:
        theta: Turning angle
    """
    vec_ab = np.array([pos_b[0] - pos_a[0] , pos_b[1] - pos_a[1] ])
    norm_ab = np.linalg.norm(vec_ab)

    vec_bc = np.array([pos_c[0] - pos_b[0] , pos_c[1] - pos_b[1] ])
    norm_bc = np.linalg.norm(vec_bc)

    dot_p =np.dot(vec_ab , vec_bc)

    cos_theta = dot_p/(norm_ab*norm_bc+np.finfo(float).eps)

    # Angle orientation
    cross_p = np.cross(vec_ab, vec_bc)
    orient = np.sign(cross_p)
    if orient == 0:
        orient = 1

    theta = np.arccos(np.around(cos_theta,4)) * orient
    return theta



########################Function to automate Turning angles processes #########
def turning_angles_producer(motion_df):
  """
  Arguments:
    motion_df: dataframe with coordinates that represents a certain type of displacement.
  Returns:
    dataframe with turning angles for each step.
  """
  trn_angle_df = pd.DataFrame(columns = ['TA_CRW'])

  for i in range(motion_df.shape[0]-2):
    ta = turning_angle(motion_df.iloc[i], motion_df.iloc[i+1], motion_df.iloc[i+2])
    tmp_df = pd.DataFrame([{'TA_CRW': ta }])
    trn_angle_df = pd.concat([trn_angle_df, tmp_df], ignore_index=True)

  return trn_angle_df


 ############# Function to get the Step Length metrics of any walk #############
def step_length_determiner(df):
  working_df = df.copy()

  # Obtention of angles through vectorization.
  vectorized_dataframe = angle_vectorizer(df)

  # Addition of angle column to original dataframe.
  working_df['Angle'] = vectorized_dataframe['Angle']

  #
  step_lengths_df = pd.DataFrame(columns = ['count','Turn_angle','Step_length'])
  count = 1
  steps= 1
  for i in range(0, working_df.shape[0] -1):
    if working_df.Angle[i] == 0.00:
      steps += 1
    else:
      tmp_df = pd.DataFrame([{'count': count ,
                              'Turn_angle':working_df.Angle[i],
                              'Step_length': steps}])
      step_lengths_df = pd.concat([step_lengths_df,tmp_df],ignore_index=True)
      count +=1
      steps = 1

  return step_lengths_df




##################### Function to create levy flights ##########################

def levy_2d(n_steps=1000,speed=1, s_x_pos=0, s_y_pos=0, cauchy_coeff=0.3, levy_coeff=1.1):
  # Determination of number of steps in a certain direction
  generated_steps = 0
  same_direction_steps = np.empty((1,1), dtype=int)
  vindex = 0
  correction_factor = 0
  while generated_steps <= n_steps:
    xstep_num = abs(levy_stable.rvs(levy_coeff, 0, loc=1,size=1))
    if xstep_num <= 1 :
      step_num = 1
    else:
      step_num = np.around(xstep_num)
    if vindex == 0:
      same_direction_steps[0] = step_num
      vindex = 1
    else:
      same_direction_steps = np.append(same_direction_steps, step_num)

    generated_steps = same_direction_steps.sum()

  correction_factor = generated_steps - n_steps
  same_direction_steps[-1]= same_direction_steps[-1] - correction_factor

  # Initialization of Brownian Walker Matrix with pandas
  BM_2d_df = pd.DataFrame(columns=['x_pos','y_pos'], dtype=np.float32)
  tmp_df = pd.DataFrame([{'x_pos':s_x_pos, 'y_pos': s_y_pos}])
  BM_2d_df = pd.concat([BM_2d_df,tmp_df], ignore_index=True)

  # Brownian walker in motion and assignation of directions
  index_count=0
  for direction in same_direction_steps:
    velocity = Vec2d(speed,0)
    # Determination of angle using Cauchy distribution
    turn_angle = abs(wrapcauchy.rvs(cauchy_coeff))
    velocity = velocity.rotated(turn_angle)
    x_velocity = velocity.x
    y_velocity = velocity.y

    for step in range(0,int(direction)):
      # Motion of Brownian walker
      tmp_df = pd.DataFrame([{'x_pos':BM_2d_df.x_pos[index_count]+x_velocity,
                              'y_pos':BM_2d_df.y_pos[index_count]+y_velocity}])
      BM_2d_df = pd.concat([BM_2d_df,tmp_df], ignore_index=True)
      index_count += 1

  return BM_2d_df

# Function that takes a walk dataframe and arrange it to apply vectorization on it ####

def angle_vectorizer(df):
  # Creation of dataframes with original column and two more shifted columns to follow a vectorization strategy
  x = df.iloc[:-2]
  x1 = pd.DataFrame(x.iloc[:])
  original_column = x1.reset_index()
  y = df.iloc[1:-1]
  y1= pd.DataFrame(y.iloc[:])
  shifted1_column = y1.reset_index()
  z= df.iloc[2:]
  z1= pd.DataFrame(z.iloc[:])
  shifted2_column = z1.reset_index()
  column_list = [original_column, shifted1_column, shifted2_column]

  # Creation of dataframe with shifted positions in columns.
  vectorized_dataframe = pd.DataFrame()
  for index, column in enumerate(column_list):
    vectorized_dataframe[f'pos_{index}'] = column.apply(lambda row: (row['x_pos'], row['y_pos']), axis=1)

  vectorized_dataframe['Angle']= vectorized_dataframe.apply(lambda row:turning_angle2(row['pos_0'],row['pos_1'], row['pos_2']), axis=1 )

  # Adding the last two rows to the vectorized_dataframe
  for i in range(0,2):
    tmp_df = pd.DataFrame([{'pos_0': (0.01,0.01) ,
                            'pos_1': (0.01, 0.01),
                            'pos_2': (0.01, 0.01),
                            'Angle': 0}])
    vectorized_dataframe = pd.concat([vectorized_dataframe,tmp_df],ignore_index=True)

  return vectorized_dataframe



# DASHBOARD

In [4]:

# Main Scaffold using box
column_a = pn.Column('Row_A', 'Row_B', 'Row_C')
row_a = pn.Row(column_a, 'Columna_B', 'Columna_C')



In [5]:
# Creation of parameters
s_rw_type ='### RW Type'
walks_radio_btn = pnw.RadioButtonGroup(name='Trajectories', value='BM' , options=['BM', 'CRW', 'LF'], width=130)
s_parameters ='### Parameters'
n_steps = pnw.IntInput(name='Number of steps', value=500, step=10, start=1, end=1000, width=130)
speed = pnw.IntInput(name='Speed', value=1, step=1, start=1, end=10, width=130)
s_x_pos = pnw.IntInput(name='Starting pos X', value=0, step=1, start=-100, end=100, width=130)
s_y_pos = pnw.IntInput(name='Starting pos y', value=0, step=1, start=-100, end=100, width=130)
cauchy_coeff = pnw.FloatSlider(name='Cauchy coefficient', value=0.1, step=0.01, start=0.01, end=0.99, width=130)
levy_coeff = pnw.FloatSlider(name='Levy coefficient', value=1.1, step=0.01, start=1.00, end=2.00, width=130, visible=True)
s_metrics = '### Metrics'
metrics_type = pnw.Select(name='Metrics Selection', value='MSD',options=['PL','MSD'], width=130)


In [ ]:


def walk_selector(selector):
  return selector
walk_option = pn.bind(walk_selector, selector=walks_radio_btn)

def walk_creator(wk,steps,spd, x_start, y_start, c_coeff, l_coeff):
  if wk =='BM':
    cauchy_coeff.visible = False
    levy_coeff.visible = False
    return bm_2d(steps, spd, x_start, y_start)
  elif wk == 'CRW':
    cauchy_coeff.visible = True
    levy_coeff.visible = False
    return crw_2d(steps, spd, x_start, y_start, c_coeff)
  elif wk =='LF':
    cauchy_coeff.visible = True
    levy_coeff.visible = True
    return levy_2d(steps, spd, x_start, y_start, c_coeff, l_coeff)
bound_walk = pn.bind(walk_creator,wk=walk_option, steps=n_steps,spd=speed,x_start=s_x_pos,
                      y_start=s_y_pos, c_coeff=cauchy_coeff, l_coeff=levy_coeff)


def walk_plotter(df, spd, steps):
  BM_df = df
  fig_BM_2d = go.Figure()
  bw_label = f''' steps {steps} speed {spd} '''
  fig_BM_2d = go.Figure(data=[go.Scatter3d(x = BM_df['x_pos'],
                                y = BM_df['y_pos'],
                                z = np.linspace(0,1,steps),
                                mode = 'lines',
                                name = bw_label,
                                showlegend = True )])
  return fig_BM_2d
bound_graph = pn.bind(walk_plotter,df=bound_walk,spd=speed,steps=n_steps)


def metrics_plotter(df, metrics_option):
  if metrics_option =='MSD':
    walk_df = smd(df)
    # Plot smd
    fig_path_smd = go.Figure()
    fig_path_smd.add_trace(go.Scatter(
        x = np.arange(0,walk_df.shape[0]),
        y = walk_df.MSD,
        marker = dict(size=2),
        line = dict(width=2),
        mode='lines',
        name='Mean Squared Displacement',
        showlegend=True))

    return fig_path_smd

  elif metrics_option == 'PL':
    dis_df = np.array([my_distance_function(df.iloc[i-1],df.iloc[i]) for i in range(1,df.shape[0])])
    pl_df = np.cumsum(dis_df)
    # Plot trajectory
    fig_path_length = go.Figure()
    fig_path_length.add_trace(go.Scatter(
        x = np.arange(len(pl_df))+1,
        y = pl_df,
        marker = dict(size=2),
        line = dict(width=2),
        mode='lines',
        name='Path Length',
        showlegend=True))

    return fig_path_length

bound_metrics = pn.bind(metrics_plotter,df=bound_walk,metrics_option=metrics_type)

column_a = pn.Column(s_rw_type, walks_radio_btn, s_parameters, n_steps,speed,
                     s_x_pos, s_y_pos,cauchy_coeff, levy_coeff, s_metrics,
                     metrics_type,)

row_a = pn.Row(column_a, bound_graph, bound_metrics)

row_a